In [1]:
from pydantic import BaseModel, Field
import openai
import base64
import openai

In [2]:
client = openai.OpenAI(
    api_key="101119258637828873013_ab4cd5a0ba83e51a",
    base_url="https://chat.maritaca.ai/api",
)

In [3]:
with open("data/2672116_67600.40752.950609.63354619.pdf", "rb") as f:
    file_content = f.read()
    file_content_base64 = base64.b64encode(file_content).decode("utf-8")

In [4]:
file_content_base64

'...'

In [5]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "file",
                "file": {
                    "filename": "2672116_67600.40752.950609.63354619.pdf",
                    "file_data": f"data:application/pdf;base64,{file_content_base64}",
                },
            },
            {
                "type": "text",
                "text": "Do que se trata esse documento",
            },
        ],
    }
]

In [6]:
response = client.chat.completions.create(
    model="sabia-3.1",
    messages=messages,
)

In [7]:
response.choices[0].message.content

'Este documento é um laudo de hemograma completo, que é um exame de sangue realizado para avaliar a saúde geral do paciente. O hemograma completo mede a quantidade e a qualidade dos três principais tipos de células no sangue: hemácias (glóbulos vermelhos), leucócitos (glóbulos brancos) e plaquetas.\n\nO laudo apresenta os resultados do exame de sangue de um paciente chamado Tiago da Silva Ferreira, incluindo os valores obtidos para hemácias, hemoglobina, hematócrito, volume corpuscular médio (VCM), hemoglobina corpuscular média (HCM), concentração de hemoglobina corpuscular média (CHCM), e o índice de anisocitose eritrocitária (RDW). Além disso, inclui o leucograma, que detalha a contagem total e diferencial de leucócitos, e a série plaquetária, que informa a contagem e o volume médio das plaquetas (VMP).\n\nObservações importantes no laudo incluem eosinofilia, que é um aumento no número de eosinófilos, um tipo de leucócito, e plaquetopenia, que é uma contagem baixa de plaquetas.\n\nAl

In [16]:
class Indicador(BaseModel):
    nome_indicador: str
    valor: float = Field(..., description="Valor (geralmente numérico) da mensuração. Não inclua a unidade de medida aqui.")
    unidade: str = Field(..., description="Unidade de medida da mensuração, como %, ml, cm, g/dL, milhões/mm3 etc. Evite caracteres extras — utilize somente o necessário.")

class Exame(BaseModel):
    tipo_exame: str
    resultados: list[Indicador]

class ResultadosExame(BaseModel):
    nome_paciente: str
    data_exame: str
    exames: list[Exame]

In [17]:
completion = client.beta.chat.completions.parse(
    model="sabia-3",
    messages=[
        {"role": "system", "content": "Extraia detalhes do do resultado dos exames."},
        {
            "role": "user",
            "content": [
                {
                    "type": "file",
                    "file": {
                        "filename": "2672116_67600.40752.950609.63354619.pdf",
                        "file_data": f"data:application/pdf;base64,{file_content_base64}",
                    },
                },
                {
                    "type": "text",
                    "text": "Quais foram os resultados do exame",
                },
            ],
        },
    ],
    response_format=ResultadosExame,
)

In [18]:
r = completion.choices[0].message.parsed

In [19]:
print(r.model_dump_json(indent=2))

{
  "nome_paciente": "TIAGO DA SILVA FERREIRA",
  "data_exame": "04/03/2024",
  "exames": [
    {
      "tipo_exame": "HEMOGRAMA COMPLETO",
      "resultados": [
        {
          "nome_indicador": "HEMÁCIAS",
          "valor": 5.21,
          "unidade": "milhões/mm3"
        },
        {
          "nome_indicador": "HEMOGLOBINA",
          "valor": 14.6,
          "unidade": "g/dL"
        },
        {
          "nome_indicador": "HEMATÓCRITO",
          "valor": 43.0,
          "unidade": "%"
        },
        {
          "nome_indicador": "VCM",
          "valor": 82.5,
          "unidade": "fl"
        },
        {
          "nome_indicador": "HCM",
          "valor": 28.0,
          "unidade": "pg"
        },
        {
          "nome_indicador": "CHCM",
          "valor": 34.0,
          "unidade": "g/dL"
        },
        {
          "nome_indicador": "RDW",
          "valor": 13.8,
          "unidade": "%"
        },
        {
          "nome_indicador": "LEUCÓCITOS",
    